In [ ]:
import numpy as np
np.float_ = np.float64

In [ ]:
from vanna.ollama import Ollama
from vanna.chromadb import ChromaDB_VectorStore

In [ ]:
class MyVanna(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

vn = MyVanna(config={'model': 'mistral'})

In [ ]:
vn.connect_to_duckdb(url=':memory:')

In [ ]:
vn.run_sql("""
CREATE TABLE items (
    id INTEGER,
    name VARCHAR,
    price DOUBLE
)
""")

In [ ]:
vn.run_sql("INSERT INTO items VALUES (1, 'apple', 0.5), (2, 'banana', 0.3), (3, 'cherry', 1.2)")

In [ ]:
vn.run_sql("SELECT * FROM items")


In [ ]:
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

In [ ]:
vn.train(documentation="Our business defines price is a positive number. The price of the smoothie is the sum of the prices of the items in the smoothie."
)


In [ ]:
vn.train(
    question="What is the price of an banana and apple smoothie?", 
    sql="SELECT SUM(PRICE) FROM ITEMS WHERE NAME IN ('banana', 'apple')",
)

In [ ]:
vn.ask(question="What is the price of an apple?", visualize=False)

In [ ]:
vn.ask(question="What is the price of an banana and apple smoothie?", visualize=False)

In [ ]:
vn.ask(question="What is the price of an apple and cherry smoothie?", visualize=False, allow_llm_to_see_data=True, auto_train=False)

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn, allow_llm_to_see_data=True)
app.run()